## RAG 시나리오(문서 파일)

이 데모는 Oracle DB에 있는 hwp, pdf 문서로 RAG와 RAG 없는 경우를 비교하는 데모임. 

LLM 서비스는 로컬 Ollama 서비스, LLAMA3.2 2b와 lanchain 프롬프트, plsql 방식을 사용합니다.

- 1.시나리오1: LLM에 직접 질문&답.
- 2.시나리오2: [RAG]DB 데이터의 벡터검색 결과를 LLM이 참조한 답변으로 lanchain과 plsql 방식 2가지가 포함됨.
- 2개의 시나리오 결과를 비교해보세요

In [13]:
# 0. 파이썬 라이브러리 로딩 및 오라클 DB 접속
# 파이썬 라이브러리 로딩 및 오라클 DB 접속
import oracledb
import oml
import re, os, time
import pandas as pd
import import_ipynb
from IPython.display import display
from user_functions import get_conn_cursor,get_db_version,exec_sql,exec_sql_rows,dml_sql,ddl_sql,clean_text1,clean_text2

# DB 접속
user_id = 'labadmin'
pwd = 'labadmin'
dbconn = 'dbserver26ai:1521/freepdb1'

conn, cursor = get_conn_cursor(user_id,pwd,dbconn)

# DB 버전 확인
get_db_version(cursor)

,PRODUCT,VERSION_FULL
0,Oracle AI Database 26ai Free,23.26.0.0.0


## 1. RAG DB 데이터 준비
- 문서(소프트웨어분리발주메뉴얼)->텍스트 추출->청킹->임베딩 실행
- 임베딩 정보를 테이블(rag_tbl_v)에 저장, 벡터 인덱싱 생성
- 참고 : 시나리오에 사용할 문서

In [2]:
# pdf 문서(소프트웨어 분리발주 매뉴얼)디스플레이
import os
from IPython.display import FileLink
img_dir = "./data/doc/rag/"  
img_file = "소프트웨어_분리발주_매뉴얼_개정배포_16.7.pdf" 
img_path = os.path.join(img_dir, img_file)
display(FileLink(img_path))

/home/labadmin/labs/vector/similarity_search/data/doc/rag/소프트웨어_분리발주_매뉴얼_개정배포_16.7.pdf

## 임베딩 및 인덱싱

문서 테이블(rag_tbl), 청킹 및 벡터 저장 테이블(rag_tbl_v), 인덱싱(HNSW)


In [3]:
# 벡터 테이블 정리
dr_tbl = """truncate table rag_tbl_v """
ddl_sql(cursor,dr_tbl)
print("테이블이 정리 됐습니다")

# 테이블 저장 문서에서 텍스트 추출 및 청킹
# 임베딩 저장(ITAS) 및 인덱싱
print("임베딩 및 인덱싱을 진행중입니다 ......")
cr_tbl = """
INSERT INTO rag_tbl_v (문서번호, 청크ID, 문서내용, 벡터)
SELECT dt.id AS 문서번호,
         et.embed_id AS 청크ID,
         et.embed_data AS 문서내용,
         to_vector(et.embed_vector) AS 벡터
  FROM rag_tbl dt,
       dbms_vector_chain.utl_to_embeddings(
         dbms_vector_chain.utl_to_chunks(
           dbms_vector_chain.utl_to_text(dt.doc),
           JSON('{"by":"words","max":"200","overlap":"10","split":"recursively","language":"korean","normalize":"all"}')
         ),
         JSON('{"provider":"database","model":"MULTILINGUAL_E5_SMALL"}')
       ) t,
       JSON_TABLE(
         t.column_value, '$'
         COLUMNS (embed_id NUMBER, embed_data VARCHAR2(4000), embed_vector CLOB)
       ) et
  WHERE dt.id = 3
"""
start_time = time.time()
rows = ddl_sql(cursor,cr_tbl)
execute_time = time.time() - start_time
print("임베딩 완료 시간 :",execute_time)

# 벡터 인덱싱 정리
ddl_sql(cursor,"drop index if exists rag_tbl_v_idx")

# 벡터 인덱싱
crIdx = """ 
CREATE VECTOR INDEX rag_tbl_v_idx on rag_tbl_v(벡터)
ORGANIZATION INMEMORY NEIGHBOR GRAPH 
WITH DISTANCE COSINE
"""
ddl_sql(cursor,crIdx)
print("인덱싱 완료")
print("벡터 데이터 준비가 됐습니다.")

테이블이 정리 됐습니다
임베딩 및 인덱싱을 진행중입니다 ......
임베딩 완료 시간 : 9.302490949630737
인덱싱 완료
벡터 데이터 준비가 됐습니다.


## 공통질문
- 1. 자연거 공통질문
- 2. Oracle-Ollama APIs
- 3. Oracle Vectory qyery

In [4]:
# 공통 질문
user_q = "공공기관 소프트웨어 분리발주 대상 사업에는 어떤 것들이 있나요?"
print("1.공통질문 :",user_q)

# Oracle's API for ollama
gen_param = """{"provider":"ollama", "host":"local", "url":"http://service-ollama:11434/api/generate", "model":"llama3.2", "temperature":"0"}"""
print("2.Oracle's API for ollama :")
print(gen_param)

# Vector query
sim_search = """
SELECT T.문서번호,T.청크ID,T.문서내용
FROM rag_tbl_v T
ORDER BY vector_distance(T.벡터, vector_embedding(MULTILINGUAL_E5_SMALL USING :q AS data),cosine)
FETCH EXACT FIRST 10 ROWS ONLY
"""

# sim_search 
"""
SELECT T.문서번호,T.청크ID, T.문서내용
FROM rag_tbl_v T
ORDER BY vector_distance(T.벡터,
                         dbms_vector.utl_to_embedding(:q, 
                                     json('{"provider":"database", 
                                            "model":"MULTILINGUAL_E5_SMALL"}')), cosine)
FETCH FIRST 10 ROWS ONLY
"""
print("3.벡터쿼리 :",sim_search)

1.공통질문 : 공공기관 소프트웨어 분리발주 대상 사업에는 어떤 것들이 있나요?
2.Oracle's API for ollama :
{"provider":"ollama", "host":"local", "url":"http://service-ollama:11434/api/generate", "model":"llama3.2", "temperature":"0.1","transfer_timeout":"300" }
3.벡터쿼리 : 
SELECT T.문서번호,T.청크ID,T.문서내용
FROM rag_tbl_v T
ORDER BY vector_distance(T.벡터, vector_embedding(MULTILINGUAL_E5_SMALL USING :q AS data),cosine)
FETCH EXACT FIRST 10 ROWS ONLY



## 2. LLM 직접 질의 답변
- LLM : LLAMA3.2
- LLM에 보강 프롬프트없이 질문합니다.

In [5]:
# LLM(llama3.2) 보강 프롬프트없이 질문합니다.
# query to llama using plsql code

gen_q = """SELECT dbms_vector.utl_to_generate_text(:txt, json(:param))"""

print("LLAMA3.2에 질문 : ",user_q," 에 대한 답변을 준비중입니다......")

import time
start_time = time.time()
result = exec_sql_rows(cursor,gen_q,{"param":gen_param,"txt":user_q})
execute_time = time.time() - start_time
lob_result = result[0][0]
print("LLAMA3.2가 학습된 정보를 기초로 답변한 내용은 다음과 같습니다.")
print("---------------------------------------------------------------------")
print(lob_result.read())
print("---------------------------------------------------------------------")
print("실행 시간 :",execute_time)

LLAMA3.2에 질문 :  공공기관 소프트웨어 분리발주 대상 사업에는 어떤 것들이 있나요?  에 대한 답변을 준비중입니다......
LLAMA3.2가 학습된 정보를 기초로 답변한 내용은 다음과 같습니다.
---------------------------------------------------------------------
공공기관의 소프트웨어 분리를 위한 발주 대상 사업에는 다음과 같은 것이 있습니다.

1. **기능별 소프트웨어 분리**: 공공기관에서 사용하는 소프트웨어를 기능별로 분리하여 개발하고 관리하는 프로젝트입니다.
2. **모듈화된 소프트웨어**: 소프트웨어를 모듈화하여 분리하여 개발하고 관리하는 프로젝트입니다.
3. **microservices**: 소프트웨어를 microservices로 구현하여 분리하여 개발하고 관리하는 프로젝트입니다.
4. **서브시스템 분리**: 공공기관의 서브시스템을 분리하여 개발하고 관리하는 프로젝트입니다.
5. **데이터 분리**: 공공기관에서 사용하는 데이터를 분리하여 개발하고 관리하는 프로젝트입니다.

이러한 사업은 공공기관에서 소프트웨어를 분리하여 개발하고 관리하는 데 필요한 기술과 프로세스를 제공합니다.
---------------------------------------------------------------------
실행 시간 : 12.094826459884644


## 3. RAG 실행
- 1.lanchain 사용 경우, 2.Oracle PLSQL 사용 경우
- RAG 실행 순서는 1. 사용자 질의가 들어 있는 문장에 대한 DB 벡터 검색, 2. 벡터 검색 결과를 참조하는 프롬프트 생성, 3. 프로프트를 LLM에 전달, 답변 요청, 4. LLM 답변
- LLM : LLAMA3.2

### 3-1. lanchain 라이브러리 사용 케이스

In [7]:
# RAG 답변
# 1. lanchain 라이브러리 사용 케이스
from langchain_core.prompts import ChatPromptTemplate
from langchain_ollama.llms import OllamaLLM

print(user_q,"에 대한 답변을 DB문서를 참조해서 답변을 준비중입니다......")

# 1. vector search
simq_start_time = time.time()
sim_result = exec_sql(cursor,sim_search,{"q":user_q})
sim_result = clean_text1(sim_result)
sim_result['문서내용']=sim_result['문서내용'].str.replace('�', '', regex=False)
simq_time = time.time() - simq_start_time

print("벡터쿼리 실행 시간 ",simq_time)
#print(sim_result)

user_question = user_q

## 2. 프롬프트 생성 
template = """
제시된 CONTEXT 정보만을 근거로,
- '답변'
- '참조문서'
로 구분하여 500자 이내로 작성하십시오. '참조문서'에는 반드시 문서번호와 청크ID를 명시하십시오.

CONTEXT:
{context}

QUESTION:
{question}
"""
#print("프롬프트 : \n", template)

prompt = ChatPromptTemplate.from_template(template)
print("langchain-ollama 프롬프트를 생성하였고, LLAMA3.2가 답변을 생성중입니다.......")

# 3. LLM 답변 생성 using lanchain_ollama
model = OllamaLLM(model="llama3.2", max_new_tokens= 1000 ,temperature=0, base_url='http://service-ollama:11434')
chain = prompt | model 

rag_start_time = time.time()
answer = chain.invoke({"context":sim_result,"question": user_question})
rag_time = time.time() - rag_start_time

print("LLM 답변 생성시간 : ",rag_time)
print("-----------------------------------------------------------------------------------")
print(answer)
print("-----------------------------------------------------------------------------------")

공공기관 소프트웨어 분리발주 대상 사업에는 어떤 것들이 있나요? 에 대한 답변을 DB문서를 참조해서 답변을 준비중입니다......
벡터쿼리 실행 시간  0.006928920745849609
langchain-ollama 프롬프트를 생성하였고, LLAMA3.2가 답변을 생성중입니다.......
LLM 답변 생성시간 :  9.629454612731934
-----------------------------------------------------------------------------------
**답변**

공공기관 소프트웨어 분리발주 대상 사업은 다음과 같습니다.

1.  소프트웨어제품이기존정보시스템이나새롭게구축하는정보시스템과통합이불가능하거나현저한비용을가진 사업
2.  5천만원 이상의 소프트웨어 다량 구매

**참조문서**

*   "분리발주 대상 사업"은 제2조에따른 총사업규모미만이거나제3조제1항에따란 사업으로 정의됩니다.
*   공공기관 소프트웨어 분리발주 대상 사업은 5천만원 이상의 소프트웨어 다량 구매로 정의됩니다.
-----------------------------------------------------------------------------------


### 3-2. Oracle plsql 사용 케이스

In [10]:
# 사용자 지정 문서를 참조한 RAG 답변
# plsql 사용 케이스
print("plsql 코드를 사용한 RAG 케이스입니다")

# Context 변수 정의
context = ""

# 쿼리 실행 context 생성
sim_result = cursor.execute(sim_search, q=user_q)

for doc_no, chunk_id, content in cursor:
    cleaned = clean_text2(content)
    # 기존 context 누적 로직 유지
    context += f"\n---\n[문서번호: {doc_no} | 청크ID: {chunk_id}]\n{cleaned}"
    
#print("Context 출력:", context)
print("context를 만들었습니다.")
    
# LLM 프롬프트 생성
indicator = """
제시된 CONTEXT 정보만을 근거로,
- '답변'
- '참조문서'
로 구분하여 500자 이내로 작성하십시오. '참조문서'에는 반드시 문서번호와 청크ID를 명시하십시오.
""" 
prompt = (indicator + "question: " + user_q + "context: " + context )

#display(prompt)

# PL/SQL 블록 (output을 바인드로 받아오기)
plsql = """
DECLARE
  input  CLOB;
  params CLOB;
  output CLOB;
BEGIN
  input := :prompt;
  params := :params;

  output := DBMS_VECTOR_CHAIN.UTL_TO_GENERATE_TEXT(input, json(params));

  :output := output;

  IF output IS NOT NULL THEN
    DBMS_LOB.FREETEMPORARY(output);
  END IF;

EXCEPTION
  WHEN OTHERS THEN
    :output := 'ERROR: ' || SQLERRM || ' (SQLCODE=' || SQLCODE || ')';
END;
"""

# 출력 바인드 변수 준비 (CLOB)
out_var = cursor.var(oracledb.DB_TYPE_CLOB)

# 실행
print("LLAMA3.2가 답변을 생성중입니다.......")
rag_start_time = time.time()
cursor.execute(plsql, prompt=prompt, params=gen_param, output=out_var)
rag_time = time.time() - rag_start_time
print("RAG 답변 생성시간 : ",rag_time)

# 결과 출력
result = out_var.getvalue()
print("---------------------------------------------------")
print("PLSQL를 사용한 RAG 결과 답변입니다.\n",result)
print("---------------------------------------------------")

plsql 코드를 사용한 RAG 케이스입니다
context를 만들었습니다.
LLAMA3.2가 답변을 생성중입니다.......
RAG 답변 생성시간 :  28.265876293182373
---------------------------------------------------
PLSQL를 사용한 RAG 결과 답변입니다.
 **답변**

[문서번호: 3 | 청크ID: X]

**분리발주 대상 사업 및 소프트웨어**

*   **분리발주 대상 사업**: 공공기관의 SW 사업이 총 사업 규모가 5억원 이상인 경우
*   **분리발주 대상 소프트웨어**: 제2조제3호에 따른 소フト웨어품질인증 제품 Good Software, GS 우선구매eligible produit

**분리발주 자격 조건**

*   SW 사업 총 사업 규모가 5억원 이상인 경우
*   소프트웨어 가격이 5천만원 이상 인 경우 (제1호의 경우 5천만원 미만도 포함)

**분리발주 대상 소프트웨어를 검토한 경우**

*   GS CC 행정업무용 선정 SW를 reference하십시오
*   제외사유서를 작성하여 분리발주 할 수 있으므로, 분리발주 제외 기준에 해당하는 경우 제외할 수 있습니다.

**분리발주 대상 소프트웨어의 price**

*   5천만원 이상이면 분리발주 대상; 5천만원 미만인 경우 통합 발주 가능

**분리발주 대상 소프트웨어를 분리발주하지 않고 일괄 발주할 경우**

*   분리발주 대상 소프트웨어 품목별 제외사유서의 제외사유 항목을 구체적으로 기재하여야 합니다.

**분리발주 대상 소프트웨어**

*   미래창조과학부고시 제2015 107호를 reference하십시오
---------------------------------------------------


In [11]:
# 프롬프트
print("프롬프트 : \n", template)
print("---------------------------------------------------")

프롬프트 : 
 
제시된 CONTEXT 정보만을 근거로,
- '답변'
- '참조문서'
로 구분하여 500자 이내로 작성하십시오. '참조문서'에는 반드시 문서번호와 청크ID를 명시하십시오.

CONTEXT:
{context}

QUESTION:
{question}

---------------------------------------------------


In [12]:
# 벡터 쿼리 결과자료 조회
# 벡터쿼리 실행
sim_result = cursor.execute(sim_search, q=user_q)

# 출력 
rows = []

for doc_no, chunk_id, content in cursor:
    cleaned = clean_text2(content)
    # 테이블용 row 수집
    rows.append({"문서번호": doc_no,"청크ID": chunk_id,"문서내용": cleaned})
    # 기존 context 누적 로직 유지
    context += f"\n---\n[문서번호: {doc_no} | 청크ID: {chunk_id}]\n{cleaned}"
    
# DataFrame 생성
result_df = pd.DataFrame(rows)

# Jupyter에서 표 형태로 출력
pd.set_option("display.width", 2000)
print("---------------------------------------------------")
display(result_df)
print("---------------------------------------------------")


---------------------------------------------------


,문서번호,청크ID,문서내용
0,3,7,예산 및 기금운용계획 집행지침 기획재정부 \n\n 사업유형별 지침 10 정보화 관련...
1,3,15,2 \n\n소프트웨어 제품을 직접 공급하거나 콘텐츠 제작을 직접 계약하면 해당 사업...
2,3,129,제\n\n2조 분리발주 대상 사업 \n\n \n\n소프트웨어산업\n\n진흥법 이하\...
3,3,11,체결을 요청하여야 한다 \n\n 국가기관등의 장은 제2조에 따른 총 사업 규모 미만...
4,3,49,공개경쟁 입찰\n\n 사업자 선정 순서에 따라 통합 본 사업과 분리발주 사업을 발주...
5,3,125,1개 혹은 동일 소프트웨어 다량 구매로 5천만원 이상이면 분리발주 대상이며 5천만원...
6,3,98,없는 경우에는 전문기관을 따로 지정하여 필요한 감독을 수행하게 할 수\n\n있음을 ...
7,3,128,전화 Fax \n\n 전화번호 \n\n사업자등록번호\n\n입찰공고 명\n\n직인\n...
8,3,6,분석\n\n설계\n\n소프트웨어 분리발주 메뉴얼 개정판 \n\n7\n\n \n\nN...
9,3,12,소프트웨어 제품이 기존 정보시스템이나 새롭게 구축하는 정보시스템과 통합이 불가능하거...


---------------------------------------------------


In [ ]:
# 정리
cursor.close()
conn.close()
print("DB 접속을 끊었습니다")

--------